# Simple FairGBM example on the UCI Adult dataset

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
SEED = 42

### Load data

In [3]:
from utils import load_uci_adult, split_X_Y_S_uci_adult

train_set, test_set = load_uci_adult()
X_train, Y_train, S_train = split_X_Y_S_uci_adult(train_set)
X_test, Y_test, S_test = split_X_Y_S_uci_adult(test_set);

100% [................................................................................] 5229 / 5229

###  Construct FairGBM model

In [4]:
from fairgbm import FairGBMClassifier

core_lgbm_params = {
    "n_estimators": 200,
    "random_state": SEED,
    "n_jobs": -2,
}

# Instantiate
fairgbm_clf = FairGBMClassifier(
    constraint_type="FNR",    # constraint on equal group-wise TPR (equal opportunity)
    multiplier_learning_rate=0.2,
    constraint_fnr_slack=0.05,
    **core_lgbm_params,
)

### Train model

In [5]:
%%time
# Train using features (X), labels (Y), and sensitive attributes (S)
fairgbm_clf.fit(X_train, Y_train, constraint_group=S_train)

CPU times: user 1min 58s, sys: 200 ms, total: 1min 59s
Wall time: 18.5 s


FairGBMClassifier(constraint_fnr_slack=0.05, constraint_type='FNR',
                  multiplier_learning_rate=0.2, n_estimators=200, n_jobs=-2,
                  random_state=42)

### Compute binary predictions
Note that the sensitive attributes are only used in training and not for predicting!

In [6]:
Y_test_pred = fairgbm_clf.predict(X_test)

### Compute Performance and Fairness metrics

In [7]:
from sklearn.metrics import accuracy_score
from utils import compute_fairness_ratio

acc_val = accuracy_score(Y_test, Y_test_pred)
eq_op_val = compute_fairness_ratio(Y_test, Y_test_pred, S_test, "TPR")

In [8]:
print(f"FairGBM:")
print(f"\tAccuracy: {acc_val:.1%}")
print(f"\tFairness: {eq_op_val:.1%}")

FairGBM:
	Accuracy: 87.1%
	Fairness: 96.6%
